<div class='alert-block alert-info'>
    <br>
    <h1 align="center"><b>  Lab session 4 :</b> Constraint Satisfaction Problem </h1>
    <h3 align="center">Artificial Intelligence Algorithms </h3>
    <h5 align="center">MESIIN476023</a></h5>
    <br>
</div>

#### CONTENTS

<font color='black'></front>    
* **Part I: CSP Basics throught the Map Coloring problem**
  1. Define the CSP class
  2. Solve the South America map coloring problem using the CSP approach
      - Exercice 1
  3. Variable Selection - Minimum remaining values and Degree heuristics
      - Exercice 2
  4. Domain Ordering - Least Constraining Value
      - Exercice 3  
  5. Forward Checking
  6. Constraint propagation - Arc Consistency (AC-3)
      - Exercice 4
  7. Min-Conflicts hill-climbing search for CSPs
      - Exercice 5

 </br>

<font color='black'></front>
* **Part II: Python libraries for solving CSPs**
  1. Or-tools library
  2. Example: Solving a Sudoku Puzzle with OR-Tools
  3. Exercice 6: University Course scheduling
    </br>


<h1 align="left"> <font color='darkblue'> Part I: CSP Basics throught the Map Coloring problem </font></a></h1>  

In this section, we explore the fundamentals of map coloring as a primer to understanding the basic concepts and techniques underpinning Constraint Satisfaction Problems (CSPs). We start by defining the CSP class.


In [43]:
# Import standard librairies
import sys
import copy


# Hide warnings in the matplotlib sections
import warnings
warnings.filterwarnings("ignore")

## I.1. CSP class

Below is an example of Python code that implements a Constraint Satisfaction Problem class using a basic backtracking search algorithm.

In [44]:
import time
class CSP:
    """This class describes finite-domain Constraint Satisfaction Problems.
    A CSP is specified by the following three inputs:
      - vars        A list of variables; each is atomic (e.g. int or string).
      - domains     A dict of {var:[possible_value, ...]} entries.
      - neighbors   A dict of {var:[var,...]} that for each variable lists
                      the other variables that participate in constraints.
      - constraints A function f(A, a, B, b) that returns true if neighbors
                      A, B satisfy the constraint when they have values A=a, B=b
    """

    def __init__(self, variables, domains, neighbours, constraints):
        """ Construct a CSP problem."""
        self.variables = variables
        self.domains = domains
        self.neighbours = neighbours
        self.constraints = constraints

        # Some performance metrics
        self.node_expansions = 0
        self.number_of_backtracks = 0
        self.pruned_values = 0
        self.number_of_constraint_checks = 0
        #-----
        self.start_time = None
        self.end_time = None

    def backtracking_search(self):
        """Set up to do recursive backtracking search."""
        self.start_time = time.time()  # Start the timer
        result = self.recursive_backtracking({})
        self.end_time = time.time()    # End the timer
        return result

    def recursive_backtracking(self, assignment):
        """Search for a consistent assignment for the csp.
        Each recursive call chooses a variable, and considers values for it."""



        if self.is_complete(assignment):
            return assignment

        var = self.select_unassigned_variable_MRV_degree(assignment)

        for value in self.order_domain_values_LCV(var, assignment):
            if self.is_consistent(var, value, assignment):
                self.node_expansions += 1
                assignment[var] = value
                result = self.recursive_backtracking(assignment)

                if result != None:
                    return result
                del assignment[var]
                self.number_of_backtracks += 1

        return None

    def select_unassigned_variable(self, assignment):
        "Select the variable to work on next."
        for variable in self.variables:
            if variable not in assignment:
                return variable
    
    def select_unassigned_variable_MRV_degree(self, assignment):
        "Select the variable to work on next."
        min_domain_size = float('inf')
        min_domain_variable = None
        for variable in self.variables:
            if variable not in assignment:
                if len(self.domains[variable]) < min_domain_size:
                    min_domain_size = len(self.domains[variable])
                    min_domain_variable = variable
                elif len(self.domains[variable]) == min_domain_size:
                    not_assigned_neighbours = 0
                    for neighbour in self.neighbours[variable]:
                        if neighbour not in assignment:
                            not_assigned_neighbours += 1
                    if not_assigned_neighbours > len(self.neighbours[min_domain_variable]):
                        min_domain_variable = variable
        return min_domain_variable

    def is_complete(self, assignment):
        "Check if all variables are assigned"
        for variable in self.variables:
            if variable not in assignment:
                return False
        return True

    def order_domain_values(self, variable, assignment):
        "Decide what order to consider the domain variables."
        all_values = self.domains[variable][:]
        return all_values
    
    def order_domain_values_LCV(self, variable, assignment):
        if variable not in assignment:
            counts = [(value, self.count_allowedValues(variable, value, assignment)) for value in self.domains[variable]]
            counts.sort(key=lambda x: x[1])
            return [value for value, count in counts]
        else:
            return [assignment[variable]]
       
    def count_allowedValues(self, variable, value, assignment):
        count = 0
        for neighbour in self.neighbours[variable]:
            if neighbour not in assignment:
                for neighbour_val in self.domains[neighbour][:]:
                    if self.constraints[variable](variable, value, neighbour, neighbour_val):
                        count += 1
        return count
            
                        

    def is_consistent(self, variable, value, assignment):

        if not assignment:
            return True

        for constraint in self.constraints.values():
            for neighbour in self.neighbours[variable]:
                self.number_of_constraint_checks += 1
                if neighbour not in assignment:
                    continue

                neighbour_value = assignment[neighbour]
                if not constraint(variable, value, neighbour, neighbour_value):
                    self.pruned_values += 1  # Increment pruned values if inconsistency is found
                    return False
        return True

    def display_performance(self):
        """Display the performance of the CSP solver."""
        print("Time taken: {:.8f} seconds".format(self.end_time - self.start_time))
        print("Node expansions: {}".format(self.node_expansions))
        print("Number of Backtracks: {}".format(self.number_of_backtracks))
        print("Number of Constraint Checks: {}".format(self.number_of_constraint_checks))
        print("Pruned values: {}".format(self.pruned_values))

## I.2. Solve the South America Map coloring problem using the CSP approach


Consider solving the map coloring problem for [South America](https://www.mapchart.net/americas.html) by applying the CSP framework. The problem is defined as follow:

In [45]:
def create_SouthAmerica_csp():
    CostaRica, Panama, Colombia, Venezuela, Guyana, Suriname, GuyaneFr, Ecuador, Peru, Brasil, Bolivia, Paraguay, Chile, Argentina, Uruguay = 'CostaRica', 'Panama', 'Colombia', 'Venezuela', 'Guyana', 'Suriname', 'GuyaneFr', 'Ecuador', 'Peru', 'Brasil', 'Bolivia', 'Paraguay', 'Chile', 'Argentina', 'Uruguay'
    values = ['Red', 'Green', 'Blue', 'Yellow']

    # Define the set of variables
    #variables = [CostaRica, Panama, Colombia, Venezuela, Guyana, Suriname, GuyaneFr, Ecuador, Peru, Brasil, Bolivia, Paraguay, Chile, Argentina, Uruguay]
    #variables = [Uruguay, Brasil, GuyaneFr, Bolivia, Suriname, Peru, Panama, Argentina, Guyana, Chile, CostaRica, Colombia, Ecuador, Venezuela, Paraguay]
    variables = [Uruguay, Suriname, GuyaneFr, Panama, CostaRica, Ecuador, Guyana, Chile, Bolivia, Paraguay, Venezuela, Argentina, Peru, Colombia, Brasil]

    # Define the domaine of each variable
    domains = {
        CostaRica: values[:],
        Panama: values[:],
        Colombia: values[:],
        Venezuela: values[:],
        Guyana: values[:],
        Suriname: values[:],
        GuyaneFr: values[:],
        Ecuador: values[:],
        Peru: values[:],
        Brasil: values[:],
        Bolivia: values[:],
        Paraguay: values[:],
        Chile: values[:],
        Argentina: values[:],
        Uruguay: values[:]
    }
    neighbours = {
        CostaRica : [Panama],
        Panama : [CostaRica, Colombia],
        Colombia : [Panama, Venezuela, Ecuador, Peru, Brasil],
        Venezuela : [Colombia, Guyana, Brasil],
        Guyana : [Venezuela, Suriname, Brasil],
        Suriname : [Guyana, GuyaneFr, Brasil],
        GuyaneFr : [Suriname, Brasil],
        Brasil : [GuyaneFr, Suriname, Guyana, Venezuela, Colombia, Peru, Bolivia, Paraguay, Argentina, Uruguay],
        Uruguay : [Brasil, Argentina],
        Argentina : [Uruguay, Paraguay, Chile, Bolivia],
        Chile : [Argentina, Bolivia, Peru],
        Bolivia : [Chile, Argentina, Paraguay, Peru],
        Paraguay : [Bolivia, Argentina, Uruguay, Brasil],
        Peru : [Bolivia, Chile, Brasil, Ecuador, Colombia],
        Ecuador : [Peru, Colombia]
    }

    def constraint_function(first_variable, first_value, second_variable, second_value):
        return first_value != second_value

    # Define the set of constraints
    constraints = {
        CostaRica: constraint_function,
        Panama: constraint_function,
        Colombia: constraint_function,
        Venezuela: constraint_function,
        Guyana: constraint_function,
        Suriname: constraint_function,
        GuyaneFr: constraint_function,
        Brasil: constraint_function,
        Uruguay: constraint_function,
        Argentina: constraint_function,
        Chile: constraint_function,
        Bolivia: constraint_function,
        Paraguay: constraint_function,
        Peru: constraint_function,
        Ecuador: constraint_function
    }

    return CSP(variables, domains, neighbours, constraints)

if __name__ == '__main__':
    SouthAmerica = create_SouthAmerica_csp()
    result = SouthAmerica.backtracking_search()
    if result:
        for area, color in sorted(result.items()):
            print("{}: {}".format(area, color))
        print('------------------------------------------')
        SouthAmerica.display_performance()
    else:
        print("No solution found")
    




Argentina: Green
Bolivia: Red
Brasil: Red
Chile: Blue
Colombia: Blue
CostaRica: Green
Ecuador: Red
Guyana: Blue
GuyaneFr: Blue
Panama: Red
Paraguay: Yellow
Peru: Green
Suriname: Green
Uruguay: Blue
Venezuela: Green
------------------------------------------
Time taken: 0.00000000 seconds
Node expansions: 15
Number of Backtracks: 0
Number of Constraint Checks: 686
Pruned values: 18


### **Exercise 1: Understanding the CSP code**

Demonstrate your understanding of the CSP code by answering the following questions:

1. What is returned by `create_SouthAmerica_csp()`?
>This function returns an instance of the CSP class that represents the South America map coloring problem. I contains the variables, the domains, the neighbours and the constraints of the problem.
   

2. What is returned by `backtracking_search()`?
>This function returns the result of the backtracking search algorithm. It returns a dictionary that contains the variables and their assigned values.
    
3. What is the purpose of `assignment` variable?
>The assignment variable is a dictionary that contains the variables and their assigned values. It is used to keep track of the assigned values during the backtracking search algorithm.

4. What is the purpose of `variable` variable?
>The variable variable is a string that represents the variable that is currently being assigned a value. It is used to keep track of the variable that is currently being assigned a value during the backtracking search algorithm.

5. What is the purpose of the following statement?
  >    `    for value in self.order_domain_values(variable, assignment)`
This statement iterates over the values of the domain of the variable that is currently being assigned a value. It is used to try all the possible values of the domain of the variable that is currently being assigned a value.
    
6. What would the following do?
  >    `if self.is_consistent('Colombia', 'Red', {'Panama': 'Blue', 'Venezuela' : 'green'}):
        assignment[variable] = value`
This statement checks if the assignment of the value 'Red' to the variable 'Colombia' is consistent with the current assignment. If it is, it adds the assignment of the value 'Red' to the variable 'Colombia' to the assignment dictionary.
7. In this case, what would then `assignment` be? (the value of `assignment`)
>The assignment dictionary would be {'Colombia': 'Red', 'Panama': 'Blue', 'Venezuela' : 'green'}.

8. What is the effect of `del assignment[variable]` in the `recursive_backtracking` method of the CSP class?
>This statement removes the assignment of the value to the variable from the assignment dictionary. It is used to backtrack when the assignment of the value to the variable leads to a dead-end.


## I.3. Variable Selection - Minimum remaining values and Degree heuristics


The Minimum Remaining Values (MRV) heuristic selects the variable that has the smallest number of legal values remaining in its domain. The Degree Heuristic is used to break ties when there are multiple variables with the same number of remaining values. It prefers the variable with the most constraints on remaining variables.

### **Exercise 2: Update the `select_unassigned_variable` method to include these heuristics into the CSP class.**
Addition done in the CSP class.

In [46]:
if __name__ == '__main__':
    SouthAmerica_MRVD = create_SouthAmerica_csp()
    result = SouthAmerica_MRVD.backtracking_search()
    if result:
        for area, color in sorted(result.items()):
            print("{}: {}".format(area, color))
    else:
        print("No solution found")
    print('------------------------------------------')
    SouthAmerica_MRVD.display_performance()

Argentina: Green
Bolivia: Red
Brasil: Red
Chile: Blue
Colombia: Blue
CostaRica: Green
Ecuador: Red
Guyana: Blue
GuyaneFr: Blue
Panama: Red
Paraguay: Yellow
Peru: Green
Suriname: Green
Uruguay: Blue
Venezuela: Green
------------------------------------------
Time taken: 0.00000000 seconds
Node expansions: 15
Number of Backtracks: 0
Number of Constraint Checks: 686
Pruned values: 18


**What do you observe?**
> We observe that the MRV heuristic reduces the number of node expansions and the number of backtracks. It also reduces the number of constraint checks and the number of pruned values. It also reduces the time taken to find a solution.

## I.4. Domain Ordering - Least Constraining Value

The Least Constraining Value heuristic in domain ordering for CSPs chooses the next value for a variable that imposes the fewest constraints on the remaining variables. This approach seeks to maximize flexibility in subsequent assignments and minimize potential conflicts. It gives preference to values that leave the largest number of options available for other variables.


### **Exercise 3: Enhance the CSP Class with the Least Constraining Value Heuristic Integration.**
Added the order_domain_values_LCV method to the CSP class.

To include the heuristic of the Least Constraining Value (LCV) in the backtracking algorithm, you need to modify the order_domain_values method so that it prefers the values that rule out the fewest choices for the neighboring variables in the constraint graph.

Here's how you could implement the LCV heuristic:

In [47]:
if __name__ == '__main__':
    SouthAmerica_MRVD_LCV = create_SouthAmerica_csp()
    result = SouthAmerica_MRVD_LCV.backtracking_search()
    if result:
        for area, color in sorted(result.items()):
            print("{}: {}".format(area, color))
    else:
        print("No solution found")
    print('------------------------------------------')
    SouthAmerica_MRVD_LCV.display_performance()

Argentina: Green
Bolivia: Red
Brasil: Red
Chile: Blue
Colombia: Blue
CostaRica: Green
Ecuador: Red
Guyana: Blue
GuyaneFr: Blue
Panama: Red
Paraguay: Yellow
Peru: Green
Suriname: Green
Uruguay: Blue
Venezuela: Green
------------------------------------------
Time taken: 0.00000000 seconds
Node expansions: 15
Number of Backtracks: 0
Number of Constraint Checks: 686
Pruned values: 18


## I.5. Backstraking search with Forward cheking

Forward checking involves checking ahead when we assign a value to a variable, to see if this would leave another variable with no legal values.

To integrate forward checking into the existing CSP class, we would need to enhance the recursive_backtracking method to monitor and adjust the legal values available for unassigned variables, eliminating those that conflict with the most recent assignment. Necessary modifications include:

1. Introducing a `forward_check` method to reduce the domain of adjacent variables contingent upon the current variable's assignment.
2. Refining the `recursive_backtracking` method to invoke `forward_check` subsequent to making an assignment.
3. Restore pruned values when a backtrack occurs due to encountering a dead-end.


1. Add a method that performs forward checking after a variable is assigned a value.
2. Modify the recursive_backtracking method to incorporate forward checking.


Here is the portion of code that implements forward checking.


In [48]:

import time
import copy
class CSP:
    # ... (keep the rest of the CSP class unchanged) ...
    """This class describes finite-domain Constraint Satisfaction Problems.
    A CSP is specified by the following three inputs:
      - vars        A list of variables; each is atomic (e.g. int or string).
      - domains     A dict of {var:[possible_value, ...]} entries.
      - neighbors   A dict of {var:[var,...]} that for each variable lists
                      the other variables that participate in constraints.
      - constraints A function f(A, a, B, b) that returns true if neighbors
                      A, B satisfy the constraint when they have values A=a, B=b
    """

    def __init__(self, variables, domains, neighbours, constraints):
        """ Construct a CSP problem."""
        self.variables = variables
        self.domains = domains
        self.neighbours = neighbours
        self.constraints = constraints

        # Some performance metrics
        self.node_expansions = 0
        self.number_of_backtracks = 0
        self.pruned_values = 0
        self.number_of_constraint_checks = 0
        #-----
        self.start_time = None
        self.end_time = None

    def backtracking_search(self):
        """Set up to do recursive backtracking search."""
        self.start_time = time.time()  # Start the timer
        result = self.recursive_backtracking({})
        self.end_time = time.time()    # End the timer
        return result


    def select_unassigned_variable(self, assignment):
        "Select the variable to work on next."
        for variable in self.variables:
            if variable not in assignment:
                return variable
    
    def select_unassigned_variable_MRV_degree(self, assignment):
        "Select the variable to work on next."
        min_domain_size = float('inf')
        min_domain_variable = None
        for variable in self.variables:
            if variable not in assignment:
                if len(self.domains[variable]) < min_domain_size:
                    min_domain_size = len(self.domains[variable])
                    min_domain_variable = variable
                elif len(self.domains[variable]) == min_domain_size:
                    not_assigned_neighbours = 0
                    for neighbour in self.neighbours[variable]:
                        if neighbour not in assignment:
                            not_assigned_neighbours += 1
                    if not_assigned_neighbours > len(self.neighbours[min_domain_variable]):
                        min_domain_variable = variable
        return min_domain_variable

    def is_complete(self, assignment):
        "Check if all variables are assigned"
        for variable in self.variables:
            if variable not in assignment:
                return False
        return True

    def order_domain_values(self, variable, assignment):
        "Decide what order to consider the domain variables."
        all_values = self.domains[variable][:]
        return all_values
    
    def order_domain_values_LCV(self, variable, assignment):
        if variable not in assignment:
            counts = [(value, self.count_allowedValues(variable, value, assignment)) for value in self.domains[variable]]
            counts.sort(key=lambda x: x[1])
            return [value for value, count in counts]
        else:
            return [assignment[variable]]
       
    def count_allowedValues(self, variable, value, assignment):
        count = 0
        for neighbour in self.neighbours[variable]:
            if neighbour not in assignment:
                for neighbour_val in self.domains[neighbour][:]:
                    if self.constraints[variable](variable, value, neighbour, neighbour_val):
                        count += 1
        return count
                        

    def is_consistent(self, variable, value, assignment):

        if not assignment:
            return True

        for constraint in self.constraints.values():
            for neighbour in self.neighbours[variable]:
                self.number_of_constraint_checks += 1
                if neighbour not in assignment:
                    continue

                neighbour_value = assignment[neighbour]
                if not constraint(variable, value, neighbour, neighbour_value):
                    self.pruned_values += 1  # Increment pruned values if inconsistency is found
                    return False
        return True

    def display_performance(self):
        """Display the performance of the CSP solver."""
        print("Time taken: {:.8f} seconds".format(self.end_time - self.start_time))
        print("Node expansions: {}".format(self.node_expansions))
        print("Number of Backtracks: {}".format(self.number_of_backtracks))
        print("Number of Constraint Checks: {}".format(self.number_of_constraint_checks))
        print("Pruned values: {}".format(self.pruned_values))

    def recursive_backtracking(self, assignment):
        """Search for a consistent assignment for the csp.
        Each recursive call chooses a variable, and considers values for it."""

        if self.is_complete(assignment):
            return assignment

        var = self.select_unassigned_variable(assignment)

        for value in self.order_domain_values(var, assignment):
            if self.is_consistent(var, value, assignment):
                assignment[var] = value
                self.node_expansions += 1
                # Perform forward checking and prune domains
                domains_backup = copy.deepcopy(self.domains)
                if self.forward_check(var, value, assignment):
                    result = self.recursive_backtracking(assignment)
                    if result is not None:
                        return result

                # If forward checking fails, or no solution is found, restore domains
                self.domains = domains_backup
                del assignment[var]
                self.number_of_backtracks += 1
        return None

    def forward_check(self, var, value, assignment):
        """Prune domain values that are inconsistent with var=value."""
        for neighbor in self.neighbours[var]:
            if neighbor not in assignment:
                for neighbor_val in self.domains[neighbor][:]:
                    if not self.constraints[var](var, value, neighbor, neighbor_val):
                        self.domains[neighbor].remove(neighbor_val)
                        self.pruned_values += 1
                if not self.domains[neighbor]:  # If the domain is empty, there's no valid assignment
                    return False
        return True
    
    # ... (keep the rest of the CSP class unchanged) ...

In [49]:
if __name__ == '__main__':
    SouthAmerica_MRVD_LCV_FC = create_SouthAmerica_csp()
    result = SouthAmerica_MRVD_LCV_FC.backtracking_search()
    if result:
        for area, color in sorted(result.items()):
            print("{}: {}".format(area, color))
    else:
        print("No solution found")
    print('------------------------------------------')
    SouthAmerica_MRVD_LCV_FC.display_performance()

Argentina: Blue
Bolivia: Red
Brasil: Yellow
Chile: Green
Colombia: Green
CostaRica: Green
Ecuador: Red
Guyana: Green
GuyaneFr: Green
Panama: Red
Paraguay: Green
Peru: Blue
Suriname: Red
Uruguay: Red
Venezuela: Red
------------------------------------------
Time taken: 0.00599527 seconds
Node expansions: 123
Number of Backtracks: 108
Number of Constraint Checks: 8097
Pruned values: 124


## I.6. Constraint Propagation - Arc Consistency (AC-3)

Constraint Propagation with AC-3 (Arc Consistency Algorithm #3) is an algorithm used in CSPs to reduce the search space by iteratively revising domains of variables to achieve arc consistency, ensuring that for every value in a variable's domain, there exists some consistent value in the domain of its adjacent variables.

AC-3 efficiently detects inconsistencies and prunes values that cannot be part of any solution before the actual search begins, greatly simplifying the problem.




### **Exercise 4: update the backtracking search method in the CSP class to include the Arc Consistency using the "AC-3" algorithm.**

In [50]:
# your code
# ...

import time
import copy
class CSP:
    # ... (keep the rest of the CSP class unchanged) ...
    """This class describes finite-domain Constraint Satisfaction Problems.
    A CSP is specified by the following three inputs:
      - vars        A list of variables; each is atomic (e.g. int or string).
      - domains     A dict of {var:[possible_value, ...]} entries.
      - neighbors   A dict of {var:[var,...]} that for each variable lists
                      the other variables that participate in constraints.
      - constraints A function f(A, a, B, b) that returns true if neighbors
                      A, B satisfy the constraint when they have values A=a, B=b
    """

    def __init__(self, variables, domains, neighbours, constraints):
        """ Construct a CSP problem."""
        self.variables = variables
        self.domains = domains
        self.neighbours = neighbours
        self.constraints = constraints

        # Some performance metrics
        self.node_expansions = 0
        self.number_of_backtracks = 0
        self.pruned_values = 0
        self.number_of_constraint_checks = 0
        #-----
        self.start_time = None
        self.end_time = None

    def backtracking_search(self):
        """Set up to do recursive backtracking search."""
        self.start_time = time.time()  # Start the timer
        result = self.recursive_backtracking({})
        self.end_time = time.time()    # End the timer
        return result


    def select_unassigned_variable(self, assignment):
        "Select the variable to work on next."
        for variable in self.variables:
            if variable not in assignment:
                return variable
    
    def select_unassigned_variable_MRV_degree(self, assignment):
        "Select the variable to work on next."
        min_domain_size = float('inf')
        min_domain_variable = None
        for variable in self.variables:
            if variable not in assignment:
                if len(self.domains[variable]) < min_domain_size:
                    min_domain_size = len(self.domains[variable])
                    min_domain_variable = variable
                elif len(self.domains[variable]) == min_domain_size:
                    not_assigned_neighbours = 0
                    for neighbour in self.neighbours[variable]:
                        if neighbour not in assignment:
                            not_assigned_neighbours += 1
                    if not_assigned_neighbours > len(self.neighbours[min_domain_variable]):
                        min_domain_variable = variable
        return min_domain_variable

    def is_complete(self, assignment):
        "Check if all variables are assigned"
        for variable in self.variables:
            if variable not in assignment:
                return False
        return True

    def order_domain_values(self, variable, assignment):
        "Decide what order to consider the domain variables."
        all_values = self.domains[variable][:]
        return all_values
    
    def order_domain_values_LCV(self, variable, assignment):
        if variable not in assignment:
            counts = [(value, self.count_allowedValues(variable, value, assignment)) for value in self.domains[variable]]
            counts.sort(key=lambda x: x[1])
            return [value for value, count in counts]
        else:
            return [assignment[variable]]
       
    def count_allowedValues(self, variable, value, assignment):
        count = 0
        for neighbour in self.neighbours[variable]:
            if neighbour not in assignment:
                for neighbour_val in self.domains[neighbour][:]:
                    if self.constraints[variable](variable, value, neighbour, neighbour_val):
                        count += 1
        return count
                        

    def is_consistent(self, variable, value, assignment):

        if not assignment:
            return True

        for constraint in self.constraints.values():
            for neighbour in self.neighbours[variable]:
                self.number_of_constraint_checks += 1
                if neighbour not in assignment:
                    continue

                neighbour_value = assignment[neighbour]
                if not constraint(variable, value, neighbour, neighbour_value):
                    self.pruned_values += 1  # Increment pruned values if inconsistency is found
                    return False
        return True

    def display_performance(self):
        """Display the performance of the CSP solver."""
        print("Time taken: {:.8f} seconds".format(self.end_time - self.start_time))
        print("Node expansions: {}".format(self.node_expansions))
        print("Number of Backtracks: {}".format(self.number_of_backtracks))
        print("Number of Constraint Checks: {}".format(self.number_of_constraint_checks))
        print("Pruned values: {}".format(self.pruned_values))

    def recursive_backtracking(self, assignment):
        """Search for a consistent assignment for the csp.
        Each recursive call chooses a variable, and considers values for it."""

        if self.is_complete(assignment):
            return assignment

        var = self.select_unassigned_variable(assignment)

        for value in self.order_domain_values(var, assignment):
            if self.is_consistent(var, value, assignment):
                assignment[var] = value
                self.node_expansions += 1
                # Perform forward checking and prune domains
                domains_backup = copy.deepcopy(self.domains)
                if self.forward_check(var, value, assignment):
                    result = self.recursive_backtracking(assignment)
                    if result is not None:
                        return result

                # If forward checking fails, or no solution is found, restore domains
                self.domains = domains_backup
                del assignment[var]
                self.number_of_backtracks += 1
        return None

    def forward_check(self, var, value, assignment):
        """Prune domain values that are inconsistent with var=value."""
        for neighbor in self.neighbours[var]:
            if neighbor not in assignment:
                for neighbor_val in self.domains[neighbor][:]:
                    if not self.constraints[var](var, value, neighbor, neighbor_val):
                        self.domains[neighbor].remove(neighbor_val)
                        self.pruned_values += 1
                if not self.domains[neighbor]:  # If the domain is empty, there's no valid assignment
                    return False
        return True
    
    # ... (keep the rest of the CSP class unchanged) ...

In [51]:
if __name__ == '__main__':
    SouthAmerica_MRVD_LCV_FC_AC3 = create_SouthAmerica_csp()
    result = SouthAmerica_MRVD_LCV_FC_AC3.backtracking_search()
    if result:
        for area, color in sorted(result.items()):
            print("{}: {}".format(area, color))
    else:
        print("No solution found")
    print('------------------------------------------')
    SouthAmerica_MRVD_LCV_FC_AC3.display_performance()

Argentina: Blue
Bolivia: Red
Brasil: Yellow
Chile: Green
Colombia: Green
CostaRica: Green
Ecuador: Red
Guyana: Green
GuyaneFr: Green
Panama: Red
Paraguay: Green
Peru: Blue
Suriname: Red
Uruguay: Red
Venezuela: Red
------------------------------------------
Time taken: 0.00599766 seconds
Node expansions: 123
Number of Backtracks: 108
Number of Constraint Checks: 8097
Pruned values: 124


## I.7. Min-conflicts hill-climbing search for CSPs

The Min-conflicts algorithm is a heuristic for solving CSPs using a local search strategy. It works by choosing a random conflicted variable and then greedily assigning it the value that minimizes the number of conflicts. The Min-conflicts algorithm can be a good alternative to backtracking search when the search space is large and the solutions are dense.

### **Exercise 5: Propose an implementation that applies the CSP framework along with the Min-Conflicts hill-climbing search strategy to solve the South America map-coloring problem.**

In [52]:
import random


class CSP:
    def __init__(self, variables, domains, neighbours, constraints):
        """ Construct a CSP problem."""
        self.variables = variables
        self.domains = domains
        self.neighbours = neighbours
        self.constraints = constraints

        # Some performance metrics
        self.step_count = None
        self.conflict_count = None

        #-----
        self.start_time = None
        self.end_time = None
        
    def is_consistent(self, variable, value, assignment):

        if not assignment:
            return True

        for constraint in self.constraints.values():
            for neighbour in self.neighbours[variable]:
                if neighbour not in assignment:
                    continue

                neighbour_value = assignment[neighbour]
                if not constraint(variable, value, neighbour, neighbour_value):
                    return False
        return True

    def min_conflicts(self, max_steps=5000):
        # Initialize performance metrics
        self.start_time = time.time()
        self.step_count = 0
        self.conflict_count = 0

        # Assign initial values randomly
        assignment = {var: random.choice(self.domains[var]) for var in self.variables}

        for step in range(max_steps):
            self.step_count += 1
            conflicted = [var for var in self.variables if not self.is_consistent(var, assignment[var], assignment)]
            self.conflict_count += len(conflicted)

            if not conflicted:
                self.end_time = time.time()
                return assignment  # Solution found

            var = random.choice(conflicted)
            min_conflict_val = self.min_conflict_value(var, assignment)
            assignment[var] = min_conflict_val

        self.end_time = time.time()
        return None  # No solution found within max_steps

    # ... [existing methods] ...
    def min_conflict_value(self, var, assignment):
        # Find the value that results in the minimum number of conflicts
        counts = {val: self.count_conflicts(var, val, assignment) for val in self.domains[var]}
        min_conflicts = min(counts.values())
        possible_vals = [val for val, count in counts.items() if count == min_conflicts]
        return random.choice(possible_vals)
    
    def count_conflicts(self, var, value, assignment):
        # Count conflicts if var=value
        count = 0
        for neighbor in self.neighbours[var]:
            neighbor_val = assignment[neighbor]
            if not self.constraints[var](var, value, neighbor, neighbor_val):
                count += 1
        return count
    
    def display_min_conflicts_performance(self):
        """Display the performance of the Min-Conflicts algorithm."""
        print("Time taken: {:.8f} seconds".format(self.end_time - self.start_time))
        print("Total steps: {}".format(self.step_count))
        print("Total conflicts encountered: {}".format(self.conflict_count))


In [53]:
if __name__ == '__main__':
    SouthAmerica_MinC_HC = create_SouthAmerica_csp()
    result = SouthAmerica_MinC_HC.min_conflicts()
    if result:
        for area, color in sorted(result.items()):
            print("{}: {}".format(area, color))
            
    else:
        print("No solution found")
    print('-------------------------------------')
    SouthAmerica_MinC_HC.display_min_conflicts_performance()
    

Argentina: Yellow
Bolivia: Red
Brasil: Green
Chile: Green
Colombia: Blue
CostaRica: Blue
Ecuador: Red
Guyana: Blue
GuyaneFr: Yellow
Panama: Yellow
Paraguay: Blue
Peru: Yellow
Suriname: Red
Uruguay: Red
Venezuela: Red
-------------------------------------
Time taken: 0.00100136 seconds
Total steps: 12
Total conflicts encountered: 49


# Part II. Python libraries for solving CSPs

Python offers a range of libraries designed to solve Constraint Satisfaction Problems, each boasting unique features and capabilities. Examples include `python-constraint`, `PuLP`, `MiniZinc`, and `OR-Tools`, among others. In this section, we will specifically explore the capabilities and functionalities of OR-Tools.



## II.1. Or-tools Library

The [OR-Tools (Operations Research Tools)](https://developers.google.com/optimization/cp/cp_solver?hl=en) library developed by Google is a powerful toolkit for solving combinatorial optimization problems, including Constraint Satisfaction Problems (CSPs). OR-Tools features the CP-SAT solver — a constraint programming solver powered by a SAT solver backend. This solver typically outperforms traditional CSP solvers in speed and capability, handling larger and more intricate problems with greater efficiency. It accommodates an extensive array of [constraints](https://developers.google.com/optimization/reference/python/sat/python/cp_model), such as all-different, linear, and scheduling constraints, making it a versatile choice for complex industry-grade and research problems.

## II.2.  Example: Solving a Sudoku Puzzle with OR-Tools
Let's consider a simple constraint satisfaction problem: a Sudoku puzzle. Sudoku is a classic example of a CSP, where the task is to fill a 9x9 grid with digits so that each column, each row, and each of the nine 3x3 subgrids that compose the grid contain all of the digits from 1 to 9 without repetition.
This requires the fulfillment of all constraints concurrently. Below is a guided example of employing Google's OR-Tools library to find a solution to a Sudoku puzzle:


In [54]:
# First install OR-Tools. You need to have the OR-Tools package installed. You can install it using pip

In [55]:
# Import the CP-SAT solver from OR-Tools.
from ortools.sat.python import cp_model

# Define the initial Sudoku grid with 0 representing empty cells.
grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

# Print the initial Sudoku grid to the console.
for i in range(9):
    if i % 3 == 0 and i != 0:
        # Print a separator line after every three lines for better readability.
        print("- - - - - - - - - - - ")
    for j in range(9):
        if j % 3 == 0 and j != 0:
            # Print a vertical separator after every three numbers for readability.
            print(" | ", end="")
        # Print each number in the grid.
        print(grid[i][j], end=" ")
    print()

# Start the process of computing a solution throught CSP approach
print("\n\n------ Solution found through CSP approach and ortools library -----\n\n")

# Create a new model for our CSP.
model = cp_model.CpModel()

# Create a dictionary to hold the CSP variables for each cell in the Sudoku grid.
sudoku = {}
for i in range(9):
    for j in range(9):
        # If the cell is initially empty, its domain is 1-9; otherwise, it is a fixed value.
        if grid[i][j] == 0:
            sudoku[i, j] = model.NewIntVar(1, 9, f'cell_{i}_{j}')
        else:
            sudoku[i, j] = model.NewIntVar(grid[i][j], grid[i][j], f'cell_{i}_{j}')

# Add the constraints that all numbers in each row/column must be different.
for i in range(9):
    model.AddAllDifferent([sudoku[i, j] for j in range(9)])
    model.AddAllDifferent([sudoku[j, i] for j in range(9)])

# Add the constraint that all numbers in each 3x3 sub-grid must be different.
for i in range(0, 9, 3):
    for j in range(0, 9, 3):
        cells = []
        for di in range(3):
            for dj in range(3):
                cells.append(sudoku[i + di, j + dj])
        model.AddAllDifferent(cells)

# Create a solver instance and solve the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print the solved Sudoku grid to the console.
if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    for i in range(9):
        if i % 3 == 0 and i != 0:
            # Print a separator line for better readability of the grid.
            print("- - - - - - - - - - - -")
        for j in range(9):
            if j % 3 == 0 and j != 0:
                # Print a vertical separator within the grid.
                print(" | ", end="")
            # Print the value of each cell in the grid.
            print(solver.Value(sudoku[i, j]), end=" ")
        print()
else:
    # Notify if no solution was found for the given Sudoku puzzle.
    print("No solution found.")

5 3 0  | 0 7 0  | 0 0 0 
6 0 0  | 1 9 5  | 0 0 0 
0 9 8  | 0 0 0  | 0 6 0 
- - - - - - - - - - - 
8 0 0  | 0 6 0  | 0 0 3 
4 0 0  | 8 0 3  | 0 0 1 
7 0 0  | 0 2 0  | 0 0 6 
- - - - - - - - - - - 
0 6 0  | 0 0 0  | 2 8 0 
0 0 0  | 4 1 9  | 0 0 5 
0 0 0  | 0 8 0  | 0 7 9 


------ Solution found through CSP approach and ortools library -----


5 3 4  | 6 7 8  | 9 1 2 
6 7 2  | 1 9 5  | 3 4 8 
1 9 8  | 3 4 2  | 5 6 7 
- - - - - - - - - - - -
8 5 9  | 7 6 1  | 4 2 3 
4 2 6  | 8 5 3  | 7 9 1 
7 1 3  | 9 2 4  | 8 5 6 
- - - - - - - - - - - -
9 6 1  | 5 3 7  | 2 8 4 
2 8 7  | 4 1 9  | 6 3 5 
3 4 5  | 2 8 6  | 1 7 9 


## II.3. Exercice 6: University Course Scheduling

* **Problem Statement:**

A small university department needs to schedule rooms for its courses. Each course needs to be scheduled in a particular room at a certain time, and no two courses can be in the same room at the same time. Additionally, some courses have prerequisites, meaning they must be scheduled earlier in the day than the courses that require them.

* **Constraints:**

    * Five courses need to be scheduled: ['Math', 'Physics', 'Chemistry', 'Biology', 'Computer Science'].
    * Three rooms are available for scheduling: ['Room A', 'Room B', 'Room C'].
    * There are 4 time slots available: ['9 AM', '11 AM', '2 PM', '4 PM'].
    * Specific courses have prerequisite requirements:
        - 'Physics' can only be scheduled if 'Math' is scheduled earlier in the day.
        - 'Biology' must be after 'Chemistry'.
    * No course can be scheduled at the same time in the same room.
    
* **Objective function**
For this scheduling problem, we want to maximize the use of early time slots (9 AM, 11 AM).


* **Requirements:**
Provide a schedule that satisfies all the constraints and places as many courses as possible in the early time slots.

    * **Exercise Steps:**

        1. Use OR-Tools to construct a model for the university course scheduling problem.
        2. Identify and define the variables for the courses, rooms, and time slots.
        3. Implement the constraints for the course prerequisites. **Hint:** use [`AddImplication`](https://developers.google.com/optimization/reference/python/sat/python/cp_model#addimplication) constraint.
        4. Implement the constraints to prevent scheduling conflicts in rooms and time slots.
        5. Integrate an objective function that gives priority to scheduling courses in early time slots.
        6. Solve the model and deduce the optimal schedule.
        7. Display the final course schedule in a readable format.



In [56]:
def schedule_courses():
    # Model
    model = cp_model.CpModel()

    # Constants
    courses = ['Math', 'Physics', 'Chemistry', 'Biology', 'Computer Science']
    rooms = ['Room A', 'Room B', 'Room C']
    times = ['9 AM', '11 AM', '2 PM', '4 PM']
    num_courses = len(courses)
    num_rooms = len(rooms)
    num_times = len(times)

    # Variables
    course_vars = {}
    for course in courses:
        for room in rooms:
            for time in times:
                course_vars[(course, room, time)] = model.NewBoolVar(f'{course}_{room}_{time}')

    # Constraints

    # Each course must be scheduled exactly once
    for course in courses:
        model.Add(sum(course_vars[(course, room, time)] for room in rooms for time in times) == 1)

    # No two courses can be in the same room at the same time
    for room in rooms:
        for time in times:
            model.Add(sum(course_vars[(course, room, time)] for course in courses) <= 1)

    # Prerequisite constraints
    # Physics must be after Math
    for room in rooms:
        for time_index, time in enumerate(times):
            if time_index > 0:
                model.AddImplication(course_vars[('Physics', room, time)], 
                                     sum(course_vars[('Math', room, prev_time)] for prev_time in times[:time_index]))

    # Biology must be after Chemistry
    for room in rooms:
        for time_index, time in enumerate(times):
            if time_index > 0:
                model.AddImplication(course_vars[('Biology', room, time)], 
                                     sum(course_vars[('Chemistry', room, prev_time)] for prev_time in times[:time_index]))

    # Objective function: maximize the use of early time slots (9 AM, 11 AM)
    model.Maximize(sum(course_vars[(course, room, time)] * (1 if time in ['9 AM', '11 AM'] else 0) 
                    for course in courses for room in rooms for time in times))

    # Solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Output the schedule
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        schedule = []
        for course in courses:
            for room in rooms:
                for time in times:
                    if solver.Value(course_vars[(course, room, time)]):
                        schedule.append((course, room, time))
        return schedule
    else:
        return "No feasible solution found."

# Running the scheduling function
schedule_courses()


TypeError: NotSupported: model.GetOrMakeBooleanIndex((Math_Room A_9 AM + Math_Room A_11 AM))